In [ ]:
#importing Packages
import numpy as np 
from PIL import Image
import os, glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import zipfile
class FingerModel():

    def __init__(self):
        pass

    #initializing the image generator of test and train
    def imageGenerator(self, train_path, test_path):
        #you can tweak all the parameters i'm giving the best parameters list which i found"
        batch = 128 #Batch size of image generaor 

        train_datagenerator = ImageDataGenerator( rescale=1./255,
                                            rotation_range=10.,
                                            width_shift_range=0.1,
                                            height_shift_range=0.1,
                                            zoom_range=0.2,
                                            horizontal_flip=True
                                          )

        test_datagenerator  = ImageDataGenerator( rescale=1./255 )

        training = train_datagenerator.flow_from_directory(
                train_path,
                target_size=(150, 150),
                color_mode='grayscale',
                batch_size=batch,
                classes=['0','1','2','3','4','5'],
                class_mode='categorical'
            )

        testing = test_datagenerator.flow_from_directory(
                test_path,
                target_size=(150, 150),
                color_mode='grayscale',
                batch_size=batch,
                classes=['0','1','2','3','4','5'],
                class_mode='categorical'
            )

        return training, testing

    def modelArchitecture(self):
        #you can tweak all the parameters i'm giving the best parameters list which i found"
        """
           Here i'm using 2D Conv because we converted our dataset to gray scale images of size 150X150
        """
        model = tf.keras.models.Sequential([
                tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 1)),
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Dense(6, activation='softmax')
                ])
        print(model.summary())
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
        return model

    def trainModel(self, train_data, test_data, model):
        #you can tweak all the parameters i'm giving the best parameters list which i found"
        no_of_epochs = 10
        no_of_steps_per_epoch = 140
        no_of_validation_steps = 30
        
        history = model.fit_generator(
                  train_data,
                  steps_per_epoch = no_of_steps_per_epoch,  
                  epochs = no_of_epochs, 
                  validation_data = test_data,
                  validation_steps = no_of_validation_steps)

        model.save("my_model.h5")#saving the model for future use.

    def main(self):
        train_data_path =  "fingers/train/"
        test_data_path =  "fingers/test/"
        train_datagen, test_datagen = self.imageGenerator(train_data_path, test_data_path)
        model = self.modelArchitecture()
        self.trainModel(train_datagen, test_datagen, model)
        return "success"


if __name__=="__main__":
    obj = FingerModel()
    obj.main()
